# TP deconvolution


In [ ]:
# google collab:
!pip install git+https://github.com/jboulanger/Analysis-of-Microscopy-Images-in-Python.git

## Fonction de transfert optique

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import mean_squared_error
from mug import deconvolution
from mug import data
from mug import utils

shape = [64,256,256]
pixel_size = [100,80,80]
otf, psf = deconvolution.otf_generator(shape,pixel_size,500,1,1.3)([0,0,0,0.0])
plt.imshow(utils.slice3d(np.log(1e-6+np.fft.fftshift(psf))))
plt.axis('off')
plt.title('Point spread function');

## Creation d'une image test

In [ ]:
def blur(img, otf):
    """Blur the image with the OTF
    Parameters
    ----------
    img : image
    otf : optical transfer function (same size as the image)
    Returns
    -------
    the blurred image
    """
    # TODO
    return img

# Generate a test sample
sample = 0.1 + 1000 * data.fibers(shape,pixel_size,L=500,smooth=30)
# Simulate the image of the sample
blurred = blur(sample, otf)
# Add some noise
blurred = np.random.poisson(blurred)
# Compute the MSE between the blurred image and the original sample
mse_blurred = mean_squared_error(sample, blurred)
# Display the images
utils.show_image_list(
    [utils.mip3d(x) for x in [sample, blurred]],
    ['Sample', f'Blurred (MSE:{mse_blurred:.2f})'])

## Filtre de Wiener

In [ ]:
def deconvolve_wiener(img, otf, snr):
    """Deconvolve the image using a Wiener filter / regularized inverse filter    

    Parameters
    ----------
    data       : numpy array
    otf        : numpy array of the same size than data
    snr        : signal to noise ratio
    Returns
    -------
    estimate   : estimated image
    """
    filter = 1 # TODO
    return np.real(np.fft.ifftn(filter * np.fft.fftn(img)))

estimate_wnr = deconvolve_wiener(blurred, otf, 10)
mse_wnr = mean_squared_error(sample, estimate_wnr)
utils.show_image_list(
    [utils.mip3d(x) for x in [sample, estimate_wnr]],
    ['Sample', f'Wiener(MSE:{mse_wnr:.2f})'])

## Algorithme de Gold-Meinel

In [ ]:
def deconvolve_gold_meinel(data, otf, background=0, iterations=100, acceleration=1.3, smooth=0):
    """Deconvolve data according to the given otf using a Gold-Meinel
    algorithm
    Parameters
    ----------
    data         : numpy array
    otf          : numpy array of the same size than data
    background   : background level
    iterations   : number of iterations
    acceleration : acceleration parameter
    Returns
    -------
    estimate   : estimated image
    dkl        : Kullback Leibler divergence

    [1] R. Gold. Rapp. tech. ANL-6984. Argonne National Lab., Ill., 1964.

    """    
    epsilon = 1e-6 # a little number    
    estimate = np.maximum(data-background, epsilon)
    for k in range(iterations):
        blurred = 1 # TODO
        ratio = 1 # TODO
        estimate = estimate * 1 # TODO    
    return estimate

estimate_gm = deconvolve_gold_meinel(blurred, otf)
mse_gm = mean_squared_error(sample, estimate_gm)
utils.show_image_list(
    [utils.mip3d(x) for x in [sample, estimate_gm]],
    ['Sample', f'G-M (MSE:{mse_gm:.2f})'])


## Algorithm de Richardson-Lucy

In [ ]:
def deconvolve_richardson_lucy(data, otf, background=0, iterations=100):
    """Deconvolve data according to the given otf using a Richardson-Lucy
    algorithm
    Parameters
    ----------
    data       : numpy array
    otf        : numpy array of the same size than data
    background : background level
    iterations : number of iterations
    Returns
    -------
    estimate   : estimated image
    dkl        : Kullback Leibler divergence

    [1] W. H. Richardson, Bayesian-Based Iterative Method of Image Restoration,
        J. Opt. Soc. Am., vol. 62, no. 1, pp. 55–59, Jan. 1972,
        doi: 10.1364/JOSA.62.000055.
    [2] L. B. Lucy, An iterative technique for the rectification of observed
        distributions, The Astronomical Journal, vol. 79, p. 745, Jun. 1974,
        doi: 10.1086/111605.

    """
    epsilon = 1e-6 # a little number
    estimate = np.maximum(data-background, epsilon)
    dkl = np.zeros(iterations)
    for k in range(iterations):
        blurred = 1 # TODO
        ratio = 1 # TODO
        estimate = estimate * 1
        dkl[k] = np.mean(blurred - data + data * np.log(np.maximum(ratio, epsilon)))
    return estimate, dkl

estimate_rl = deconvolve_richardson_lucy(blurred, otf)
mse_rl = mean_squared_error(sample, estimate_rl)
utils.show_image_list(
    [utils.mip3d(x) for x in [sample, estimate_rl]],
    ['Sample', f'R-L (MSE:{mse_rl:.2f})'])

## Minimization de la variation total (TV)

In [ ]:
def deconvolve_total_variation(
        data:np.ndarray,
        otf:np.ndarray,
        background=0.,        
        regularization:float=0.5,
        max_iter:int=100,
        step_size:float=1,
        beta:float=0.1) -> np.ndarray:
    """Deconvolve the image with a total variation regularization
    Parameters
    ----------
    data       : numpy array
    otf        : numpy array of the same size than data
    background : background as float or nd.array
    ...
    Returns
    -------
    estimate   : estimated image
    """
    from scipy import ndimage
    epsilon = 1e-6    
    estimate = np.maximum(np.real(np.fft.ifftn(otf * np.fft.fftn(data-background))), epsilon)
    D = [np.array([0,-1,1]).reshape(s) for s in [[1,1,3],[1,3,1],[3,1,1]]]    
    Dstar = [np.array([-1,1,0]).reshape(s) for s in [[1,1,3],[1,3,1],[3,1,1]]]    
    Hstarf = np.real(np.fft.ifftn(np.conjugate(otf) * np.fft.fftn(data)))
    HtH = np.conjugate(otf) * otf
    for _ in range(max_iter):
        G = [ndimage.convolve(estimate, d, mode='reflect') for d in D]
        N = np.sqrt(sum([np.square(g) for g in G]) + beta)
        curv = sum([ndimage.convolve(g/N,d) for g,d in zip(G,Dstar)])
        veloc = 1 # TODO        
        estimate = np.maximum(estimate - step_size * veloc, 0)
    return estimate